In [8]:
import os
import urllib3
import numpy as np 
import pandas as pd


# Goal: acquire pubmed information regarding given search term

1. Using an ESearch API "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi" to acquire the IDs of relevant articles to query 
  * Note: queries seem to be ordered by relevance, confirm! 
2. Save XML and parse into list of ID in string 
3. Use list from above to query EFetch API and download article summaries in .XML
  * https://www.ncbi.nlm.nih.gov/books/NBK25499/table/chapter4.T._valid_values_of__retmode_and/?report=objectonly Parameters to EFetch 


# Deliverables 
1. PUBMED\_IDS\_*.xml in ds/dataset/
2. PUBMED\_ABSTRACTS\_*.xml in ds/dataset/

In [4]:
# CONSTANT DEFINITION

DATASET_PATH = "../dataset"
ESEARCH_URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
EFETCH_URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"

# Define Query parameters here 
QUERY_TERM = "hlh"
# Max is 100 000, otherwise use history 
MAX_IDS_RETURNED = 99999



In [38]:
# Use eSearch API to acquire IDs for texts related to "hlh" 

http = urllib3.PoolManager()
eSearchRes = http.request(
    'GET', 
    f'{ESEARCH_URL}?term={QUERY_TERM}&retmax={MAX_IDS_RETURNED}'
    )

with open(f'{DATASET_PATH}/PUBMED_IDS_TERM={QUERY_TERM}.xml', 'wb') as f:
    f.write(eSearchRes.data)

In [5]:
# Read from stored XML and parse IDs

import xml.etree.ElementTree as et 

xtree = et.parse(f'{DATASET_PATH}/PUBMED_IDS_TERM={QUERY_TERM}.xml')
IdNode = xtree.getroot().find("IdList")

ID_LIST = [i.text for i in IdNode.findall("Id")]

In [6]:
# Construct form-data for HTTP POST request to EFetch 

EFETCH_POST_BODY = {
    "db": "pubmed",
    "retmode": "xml",
    "id": ",".join(ID_LIST)
}

In [41]:
# Retrieve XML from EFetch API


http = urllib3.PoolManager()

eFetchRes = http.request(
    "POST",
    EFETCH_URL,
    fields = EFETCH_POST_BODY
)

with open("../dataset/PUBMED_ABSTRACTS_TERM={QUERY_TERM}.xml", 'wb') as f:
    f.write(eFetchRes.data)

In [3]:
# Parsing XML into pandas dataframe --> .csv for analysis 
import xml.etree.ElementTree as et 

titles = []
PMID = []
doi = []
keywords = []




xtree = et.parse("../dataset/hlh_PUBMED_ABSTRACTS.xml")
root = xtree.getroot()


for article in root.findall("PubmedArticle"):
    for i in article.iter("ArticleTitle"):
        print(i.text)



tic peptide.
Chemical synthesis of peptide fragments of the hormone-specific beta-subunit of human follicle-stimulating hormone.
[Ultrastructural localization of pituitary hormones by immunocytochemical reactions in ultrathin sections of adenomas].
Prolactin-mediated progesterone secretion by cultured rat granulosa cells: regulation by purified human glycoprotein hormones and their subunits.
High correlation between prolactinemia, 125-I hLH binding and progesterone secretion by an experimental luteoma.
In vitro studies of prolactin inhibition of luteinizing hormone action on Leydig cells of rats and mice.
Prostaglandin F2 alpha inhibits luteinizing hormone (LH)-induced increase in LH receptor binding to isolated rat luteal cells.
Use of a highly sensitive and specific immunoradiometric assay for detection of human chorionic gonadotropin in urine of normal, nonpregnant, and pregnant individuals.
Stimulation of sperm production by human luteinizing hormone in gonadotropin-suppressed norm